In [ ]:
# pip install -i https://pypi.tuna.tsinghua.edu.cn/simple agentscope --pre --user
# https://doc.agentscope.io/zh_CN/tutorial/103-example.html
# https://github.com/modelscope/agentscope/blob/main/README_ZH.md

In [1]:
import os
import sys
from typing import Any
import json
import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import torch as th
import agentscope
from agentscope import msghub
from agentscope.agents import (UserAgent, DialogAgent)
from agentscope.message import Msg
from agentscope.service import (ServiceToolkit, ServiceResponse)

In [2]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(th.__version__)
print(th.version.cuda)
print(agentscope.__version__)

device = cuda; devive_cnt = 1
2.2.0+cu118
11.8
0.0.6alpha2


## step-1: 准备模型配置文件

In [7]:
lst_model_config = [
    {
        "config_name": "ollama_chat-qwen2:7b",
        "model_type": "ollama_chat",
        "model_name": "qwen2:7b",
        "options": {"temperature": 1.0},
        "keep_alive": "5m",
        # "api_key": "xxx",
        # "organization": "xxx",
    }
]  # 可配置多个模型

In [8]:
agentscope.init(model_configs=lst_model_config)

2024-08-18 20:45:10.300 | WARNING  | agentscope.models:read_model_configs:179 - config_name [ollama_chat-qwen2:7b] already exists.
2024-08-18 20:45:10.302 | INFO     | agentscope.models:read_model_configs:186 - Load configs for model wrapper: ollama_chat-qwen2:7b


[]

## step-2: 连接 AgentScope

In [13]:
agent = DialogAgent(
    name="assistant",
    sys_prompt="You are a helpful assistant.",
    model_config_name="ollama_chat-qwen2:7b"
)

2024-08-18 20:46:32.246 | INFO     | agentscope.models.model:__init__:201 - Initialize model by configuration [ollama_chat-qwen2:7b]


In [14]:
msg = Msg(name="user", content="你是谁？", role="user")
res = agent(msg)

assistant: 我是一个大模型，可以回答各种问题、提供信息和进行对话。有什么我可以帮助你的吗？


In [15]:
msg

{'id': 'd7ef50340fbc4cbdabd34942e1812d52',
 'timestamp': '2024-08-18 20:46:33',
 'name': 'user',
 'content': '你是谁？',
 'role': 'user',
 'url': None,
 'metadata': None}

In [16]:
res

{'id': '7b17455dddb7465790f2b1adad37023c',
 'timestamp': '2024-08-18 20:46:36',
 'name': 'assistant',
 'content': '我是一个大模型，可以回答各种问题、提供信息和进行对话。有什么我可以帮助你的吗？',
 'role': 'assistant',
 'url': None,
 'metadata': None}

In [17]:
msg = Msg(name="user", content="你指的大模型是什么意思？", role="user")
res = agent(msg)

assistant: "大模型"通常指的是在人工智能领域中用于处理自然语言任务的复杂算法或机器学习模型。这些模型能够通过大量的数据训练来理解和生成人类语言，从而实现诸如回答问题、提供建议、解释概念、撰写文本等任务。

具体到我这个例子，“大模型”可能是指像通义千问这样的超大规模预训练模型。这类模型在处理自然语言理解与生成时具有较高的性能和能力，能够适应多种应用场景，并且通过微调可以针对特定领域或任务进行优化。我的目标是为用户提供准确、有用的信息和支持。

如果您有任何具体问题需要解答或者想要了解某个主题，请随时告诉我！我会尽力提供帮助。


In [18]:
msg = Msg(name="user", content="还有呢？", role="user")
res = agent(msg)

assistant: 当然，除了回答问题和提供信息之外，我还可以：

1. **解释概念**：对于复杂的科学、技术或学术概念，我可以尝试用简单易懂的语言进行解释。

2. **提供建议**：在某些情况下，比如学习方法、时间管理或是职业规划方面，我可以根据现有的知识和经验给出建议。

3. **撰写文本**：无论是文章、报告、故事还是邮件等，我都可以帮助生成相应的文本内容。

4. **翻译**：虽然我的能力有限，但我可以尝试将文本从一种语言翻译成另一种语言。

5. **进行对话**：与您进行自然流畅的对话，讨论各种话题，并提供相关的信息或观点。

如果您有任何具体需求或者想要了解某个特定主题，请随时告诉我。我会根据您的请求来调整回答的内容和方式，以满足您的需求。


In [21]:
agent.memory.get_memory()

[{'id': 'd7ef50340fbc4cbdabd34942e1812d52',
  'timestamp': '2024-08-18 20:46:33',
  'name': 'user',
  'content': '你是谁？',
  'role': 'user',
  'url': None,
  'metadata': None},
 {'id': '7b17455dddb7465790f2b1adad37023c',
  'timestamp': '2024-08-18 20:46:36',
  'name': 'assistant',
  'content': '我是一个大模型，可以回答各种问题、提供信息和进行对话。有什么我可以帮助你的吗？',
  'role': 'assistant',
  'url': None,
  'metadata': None},
 {'id': '7b34ed0a091f463eb76064450866e58d',
  'timestamp': '2024-08-18 20:48:35',
  'name': 'user',
  'content': '你指的大模型是什么意思？',
  'role': 'user',
  'url': None,
  'metadata': None},
 {'id': '35585e260fc54b9a98b6e80cba15baee',
  'timestamp': '2024-08-18 20:48:55',
  'name': 'assistant',
  'content': '"大模型"通常指的是在人工智能领域中用于处理自然语言任务的复杂算法或机器学习模型。这些模型能够通过大量的数据训练来理解和生成人类语言，从而实现诸如回答问题、提供建议、解释概念、撰写文本等任务。\n\n具体到我这个例子，“大模型”可能是指像通义千问这样的超大规模预训练模型。这类模型在处理自然语言理解与生成时具有较高的性能和能力，能够适应多种应用场景，并且通过微调可以针对特定领域或任务进行优化。我的目标是为用户提供准确、有用的信息和支持。\n\n如果您有任何具体问题需要解答或者想要了解某个主题，请随时告诉我！我会尽力提供帮助。',
  'role': 'assistant',
  'url': None